In [1]:
import pandas as pd
import numpy as np
import pickle
import plotly.express as px
import networkx as nx
import matplotlib.pyplot as plt 


# get the money flow between daos
Criteria: if a transaction is targeted to "x.sputnikdao.near" or "x.sputnik-dao.near", then it's a between-dao transaction.

In [2]:
fdir = 'data/'
fname = 'Sputnik DAO Proposals.xlsx'
df_dao_proposals = pd.read_excel(fdir+fname,sheet_name=None)
df_out={'dao_give':[]}
df_in={'dao_receive':[]}
df_alltrans={'dao_give':[],'dao_receive':[],'amount':[],'time':[]}
for daoname in df_dao_proposals.keys():
    if daoname=='WORKSPACE': # this is not a dao, so skip it
        continue
    df = df_dao_proposals[daoname]
    df.columns = df.columns.str.replace(' ', '_').str.lower() # clean column names to avoid pandas key error
    
    # transform the amount to be in the unit of near
    if 'kind__amount' in df.keys():
        df['amount_(near)'] = pd.to_numeric(df['kind__amount'],errors = 'coerce')/(10**24)
       
    else:
        df['amount_(near)'] = pd.to_numeric(df['amount_(near)'],errors = 'coerce')
    
    # also, standardize column names
    if 'target' not in df.columns:
        df.columns = df.columns.str.replace('.*recipient.*', 'target',regex=True)

    # look into the fund flow
    allgives = {'receiver':[],'amount':[]}
    for index, prop in df.iterrows():
        if prop['status']=='Success': # only counting successful transactions
#             if prop['target'][-17:] == '.sputnik-dao.near':
#                 dao = prop['target'][:-17]
#                 if dao[-4:]=='-dao':
#                     dao=dao[:-4]
#                 df_alltrans['dao_receive'].append( dao)
            if  prop['target'][-16:] == '.sputnikdao.near':
                dao=prop['target'][:-16]
                if dao[-4:]=='-dao':
                    dao=dao[:-4]
                dao=dao.replace('-', '') # regularize the naming conventions
                df_alltrans['dao_receive'].append(dao)
            else: 
                continue

            daolower = daoname.lower().replace(' ','').replace('-', '') # lower case version of the dao name in order to be consistent
            if daoname == 'Open Web Sandbox':
                daolower = 'sandbox'
            df_alltrans['dao_give'].append(daolower)
            df_alltrans['amount'].append( prop['amount_(near)'])
            df_alltrans['time'].append(pd.to_datetime(prop['vote_period_end']))
                
    
df_alltrans = pd.DataFrame(df_alltrans)

In [3]:
df_alltrans.head()

,dao_give,dao_receive,amount,time
0,communitysquad,createbase,300.0,2021-04-20 18:19:21.470696448
1,communitysquad,portuguese,150.0,2021-05-13 19:41:04.447497216
2,communitysquad,sandbox,2551.0,2021-05-19 15:22:09.020133120
3,communitysquad,nftgaming,2000.0,2021-05-20 16:12:46.214436608
4,communitysquad,vr,2000.0,2021-05-20 16:16:08.911319040


In [6]:
df_alltrans.to_csv('transactions_between_daos.csv')

# Question 1: how much of the funding comes from community squad?

In [7]:
daogive = 'communitysquad'
comsquad_give = df_alltrans.loc[df_alltrans['dao_give']==daogive].groupby('dao_receive').sum().sort_values(by='amount')
print(comsquad_give)

                               amount
dao_receive                          
neararabia                      43.50
createbase                     300.00
nearsamurai                    490.00
bogotahacknode                 560.00
san-francisco-near-hacknode    700.00
near_asia_dev_dao              710.00
nyc                           1000.00
near-data-center              1000.00
nft-onboarding                1005.00
lisboa-hack                   1151.95
npk-guild-nigeria             1367.50
portuguese                    1395.00
telegramcommunity             1600.00
mukachevo                     1885.00
near_india_guild              1943.50
art                           2000.00
nft-gaming                    2000.00
vr                            2000.00
misfits                       4142.00
rucommunity                   5250.00
vitalpointai                  6600.00
china_charity_dao            10000.00
vietnam_covid19_charity      10000.00
marketing-vertical           20831.14
creatives   

conclusion 1: not all. many daos like human guild, degen, NxM do not receive funding from community squad (so how to track their income?)

## for daos receiving money from community squad, is that their major source?

In [94]:
# load the current value of each dao
df_daostats = pickle.load(open('dao_stats','rb'))
df_daostats['Name']=df_daostats['Name'].str.replace('.sputnikdao.near','',regex=True)

df_daostats.head()

,Name,Ⓝ Value,Value (USD),Proposals,In Progress,Successful,Failed,Expired
0,genesis,15073.0,157965.0,45.0,0.0,15.0,9.0,21.0
1,academy,48.0,503.0,5.0,0.0,2.0,0.0,3.0
2,ethdenver,264.0,2767.0,133.0,0.0,92.0,12.0,28.0
3,arts,205.0,2144.0,8.0,0.0,7.0,0.0,1.0
4,createbase,503.0,5272.0,483.0,4.0,370.0,77.0,22.0


In [90]:
# load the total funding distributed historically
df_grants = pickle.load(open('dao_prop_stats','rb'))
df_grants.head()

,dao,grants,total_proposal,total_success_proposal,first_prop_time,last_prop_time,proposal_freq
12,Grants,1700.000000,5,1,1.616817e+18,1.621880e+18,0.085321
11,Bounties,4986.000000,8,6,1.630957e+18,1.634140e+18,0.217143
8,Genesis,5928.000000,45,15,1.610937e+18,1.625782e+18,0.261898
10,NxM,7386.751634,106,85,1.617318e+18,1.637067e+18,0.463734
2,Marketing Vertical,20724.000000,71,35,1.630059e+18,1.637152e+18,0.864883


In [96]:
bigdaos = ['Creatives','Createbase','Marketing Vertical','Open Web Sandbox']
for dao in bigdaos:
    daoalt = dao.lower().replace(' ','-')
    if dao == 'Open Web Sandbox':
        daoalt = 'sandbox'
    print("{} has total fund {}, {} comes from community squad.".format(dao,float(df_grants.loc[df_grants['dao']==dao]['grants'])+float(df_daostats.loc[df_daostats['Name']==daoalt]['Ⓝ Value']),comsquad_give.loc[daoalt]['amount']))
    

Creatives has total fund 82512.02, 62434.0 comes from community squad.
Createbase has total fund 67297.65377887999, 300.0 comes from community squad.
Marketing Vertical has total fund 21499.0, 20831.14 comes from community squad.
Open Web Sandbox has total fund 73291.17000000006, 73271.0 comes from community squad.


conclusion: some has most of its funding from community squad(marketing vertical, sandbox), some has the majority (Creatives), some only a tiny bit (Createbase)

# Question 2: if it's not from community squad, can we find all the sources of funds?

## summarize all the transactions between two daos

In [87]:
df_trans_receive = df_alltrans[['dao_receive','dao_give','amount']].groupby(['dao_receive','dao_give']).sum()
df_trans_receive.head()

amount
dao_receive   dao_give               
afrikaburn    community-squad   428.0
alas          nxm                 0.0
appalabs      human-guild      2500.0
arrozcriativo creatives        5581.0
              nxm                38.0

In [85]:
df_trans_receive.to_csv('transaction_receiver_sums.csv')

In [86]:
df_trans_receive_table=pd.read_csv('transaction_receiver_sums.csv') # after doing this, multi-level column becomes normal columns

### organize it into source-target format in json

In [82]:
json_trans=df_trans_receive_table.rename(columns={'dao_receive':'target','dao_give':'source','amount':'value'}).to_json(orient='records')
json_trans

'[{"target":"afrikaburn","source":"community-squad","value":428.0},{"target":"alas","source":"nxm","value":0.0},{"target":"appalabs","source":"human-guild","value":2500.0},{"target":"arrozcriativo","source":"creatives","value":5581.0},{"target":"arrozcriativo","source":"nxm","value":38.0},{"target":"art","source":"community-squad","value":2000.0},{"target":"art","source":"human-guild","value":2000.0},{"target":"berryclub","source":"createbase","value":150.0},{"target":"bogotahacknode","source":"community-squad","value":560.0},{"target":"c1","source":"creatives","value":3773.0},{"target":"ceramic","source":"genesis","value":333.0},{"target":"china_charity_dao","source":"community-squad","value":10000.0},{"target":"clan","source":"creatives","value":340.0},{"target":"codame","source":"createbase","value":5.0},{"target":"createbase","source":"community-squad","value":300.0},{"target":"createbase","source":"createbase","value":20.0},{"target":"createbase","source":"creatives","value":7710.

In [66]:
json_nodes=list(map(set,df_trans_receive_table[['dao_receive','dao_give']].values.T))
json_nodes= [{'name':kk } for kk in list(json_nodes[0].union(json_nodes[1]))]

In [67]:
json_nodes

[{'name': 'misfits'},
 {'name': 'genesis'},
 {'name': 'nearch4n'},
 {'name': 'degensdao'},
 {'name': 'mukachevo'},
 {'name': 'near_india_guild'},
 {'name': 'onboarding'},
 {'name': 'sandbox'},
 {'name': 'afrikaburn'},
 {'name': 'bogotahacknode'},
 {'name': 'arrozcriativo'},
 {'name': 'clan'},
 {'name': 'dj'},
 {'name': 'tb'},
 {'name': 'lisboa-node'},
 {'name': 'vitalpointai'},
 {'name': 'near_codame_china'},
 {'name': 'graffiti'},
 {'name': 'human-guild'},
 {'name': 'nearsamurai'},
 {'name': 'telegramcommunity'},
 {'name': 'move'},
 {'name': 'berryclub'},
 {'name': 'nyc'},
 {'name': 'art'},
 {'name': 'codame'},
 {'name': 'vietnam_covid19_charity'},
 {'name': 'lisboa-hack'},
 {'name': 'nxm'},
 {'name': 'move-capital'},
 {'name': 'wiki'},
 {'name': 'ceramic'},
 {'name': 'social-tokens'},
 {'name': 'ecosystem-(kog)'},
 {'name': 'motion'},
 {'name': 'rucommunity'},
 {'name': 'marmaj'},
 {'name': 'npk-guild-nigeria'},
 {'name': 'cudo'},
 {'name': 'generative-art'},
 {'name': 'portuguese'},

In [ ]:
# get full json
fulljson = 'let data2 = {"nodes": '+str(json_nodes)+',  "links": '+json_trans+ '};'

In [70]:
with open("fundingflow.js", "w") as file:
    file.write(fulljson)

## create base

In [100]:
dao = 'createbase'
comsquad_give = df_alltrans.loc[df_alltrans['dao_receive']==dao].groupby('dao_give').sum().sort_values(by='amount')
print(comsquad_give)

                 amount
dao_give               
Createbase         20.0
Community Squad   300.0
Creatives        7710.0


## creatives

In [101]:
dao = 'creatives'
comsquad_give = df_alltrans.loc[df_alltrans['dao_receive']==dao].groupby('dao_give').sum().sort_values(by='amount')
print(comsquad_give)

# note: there's money from creatives to creatives for "Moving funds from Sputnik to Astro!"

                   amount
dao_give                 
Creatives        31190.58
Community Squad  62434.00


## degens

In [105]:
dao = 'degens'
comsquad_give = df_alltrans.loc[df_alltrans['dao_receive']==dao].groupby('dao_give').sum().sort_values(by='amount')
print(comsquad_give)

Empty DataFrame
Columns: [amount]
Index: []


## NxM

In [88]:
df_trans_receive.loc['nxm'] # can't find other source of income for nxm. this is for transfering to astro, again.

,amount
dao_give,
creatives,4205.0
marketing-vertical,140.0
nxm,420.0


In [89]:
# how much of nxm dao is from creatives / marketing vertical compared to their total funds?
dao='NxM'
daoalt = dao.lower()
    
print("{} has total fund {}".format(dao,float(df_grants.loc[df_grants['dao']==dao]['grants'])+float(df_daostats.loc[df_daostats['Name']==daoalt]['Ⓝ Value'])))

# okay, still a large chunk of income missing

NameError: name 'df_grants' is not defined

conclusion: still missing a bit chunk of money source.

## in general, who are the significant money givers?

In [139]:
df_give_sum = df_alltrans.groupby(['dao_give']).sum()

In [142]:
df_give_sum.sort_values(by='amount',ascending=False)

,amount
dao_give,
Community Squad,221620.59
Creatives,70137.02
Move Capital,48001.00
Ecosystem (KoG),28900.00
Human Guild,19500.00
Createbase,7768.00
Marketing Vertical,3950.00
NxM,1266.00
Genesis,333.00


# Time progression

In [104]:
df_alltrans.sort_values(by='time').head()

,dao_give,dao_receive,amount,time
126,createbase,createbase,10.0,2021-02-22 19:52:57.994595328
127,createbase,createbase,10.0,2021-03-05 18:53:14.874822912
128,createbase,createbase,NaN,2021-03-19 16:09:30.287335168
129,createbase,berryclub,150.0,2021-04-12 04:51:54.325901056
0,community-squad,createbase,300.0,2021-04-20 18:19:21.470696448


In [4]:
# before that, clear out the transactions for astro i.e. same receiver and giver
print(len(df_alltrans))
df_cleantrans = df_alltrans.loc[df_alltrans['dao_give']!=df_alltrans['dao_receive']]
print(len(df_cleantrans))

122
119


In [9]:
df_cleantrans['time'].min()

Timestamp('2021-04-12 04:51:54.325901056')

In [10]:
df_cleantrans['time'].max()

Timestamp('2021-11-18 13:30:37.025602600')

In total we have data for a bit more than 7 months. Let's look at each month.

## total fund giver and receiver

In [5]:
df_month=df_cleantrans.groupby(pd.Grouper(key='time',freq='M')).nunique()

df_month = df_month.rename(columns={'dao_give':'total_giver','dao_receive':'total_receiver'})

df_month

,total_giver,total_receiver,amount
time,,,
2021-04-30,2,2,2
2021-05-31,3,10,8
2021-06-30,4,9,10
2021-07-31,3,4,5
2021-08-31,3,18,17
2021-09-30,6,25,26
2021-10-31,5,17,18
2021-11-30,4,16,17


In [21]:
fig1 = px.bar(df_month,y='total_giver',title='total number of DAOs receiving funds')
fig1.write_html("visualization/fig1_Nreceiver.html",full_html=False, include_plotlyjs='cdn')  
  

TypeError: write_image() got an unexpected keyword argument 'full_html'

In [23]:
fig1.write_image("visualization/fig1_Nreceiver.png")

In [8]:
px.bar(df_month,y='total_receiver',title='total number of DAOs receiving funds')

In [14]:
df_month = df_month.rename(columns={'dao_give':'total_giver','dao_receive':'total_receiver'})

In [16]:
df_month[['total_giver','total_receiver']]

,total_giver,total_receiver
time,,
2021-04-30,2,2
2021-05-31,3,10
2021-06-30,4,9
2021-07-31,3,4
2021-08-31,3,18
2021-09-30,6,25
2021-10-31,5,17
2021-11-30,4,16


## total volume of fund tranfer (between daos)

In [24]:
# visualize the total fund transfers
fig2=px.histogram(df_cleantrans,x='time',y='amount',histfunc='sum',title='total volume of fund tranfer (between daos)')
fig2.write_image("visualization/fig2_totalamount.png")

## total fund giver

### ratio of community-squad being the giver

## create network of money flow

In [124]:
df_time_flow=df_cleantrans.groupby([pd.Grouper(key='time',freq='M'),'dao_give','dao_receive']).sum()

### alterntaive visualization: tree map
easy for the grammar; good interative visualization. only problem is not able to represent more than 1 level hierarchy. 

TODO: keep the color consistent across months

In [199]:
for idx, df_month_flow in df_time_flow.groupby(level='time'): 
    df_month_flow.reset_index(inplace=True) # convert the multi-index(from grouping) into columns
    fig = px.treemap(df_month_flow, path=[ 'dao_give', 'dao_receive'], values='amount')
    fig.show()

### network visualization

network design: 
- overall layout -- maybe using hierarhical structure is better
- value is sum for each node.is someone is receiver and giver...let's only do value depending on the giver?

In [21]:
for idx, df_month_flow in df_time_flow.groupby(level='time'): 
    df_month_flow.reset_index(inplace=True) # convert the multi-index(from grouping) into columns
    df_month_flow['weight']= np.log(df_month_flow['amount'])
    df_month_flow.replace([np.inf, -np.inf], np.nan, inplace=True)  
    df_month_flow.dropna(inplace=True)
    # set the edge width
    df_month_flow['penwidth']= .5+4*(df_month_flow['weight'] - min(df_month_flow['weight']))/(max(df_month_flow['weight'])-min(df_month_flow['weight']))
    if idx.month==6:
        break

/Users/zhiweili/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning:

divide by zero encountered in log



In [23]:
for idx, df_month_flow in df_time_flow.groupby(level='time'): 
    df_month_flow.reset_index(inplace=True) # convert the multi-index(from grouping) into columns
    df_month_flow['weight']= np.log(df_month_flow['amount'])
    # Dropping all the rows with 0 funding amount values
    df_month_flow.replace([np.inf, -np.inf], np.nan, inplace=True)  
    df_month_flow.dropna(inplace=True)
    # set the edge width
    df_month_flow['penwidth']= .5+4*(df_month_flow['weight'] - min(df_month_flow['weight']))/(max(df_month_flow['weight'])-min(df_month_flow['weight']))
    
    G=nx.from_pandas_edgelist(df_month_flow, 'dao_give', "dao_receive", edge_attr=True,create_using=nx.DiGraph())
    
    
    # define node attributes
    dict_month_giver=np.log(df_month_flow.groupby('dao_give').sum()).to_dict()['amount']
    #nx.set_node_attributes(G,min(dict_month_giver),'size')
    nx.set_node_attributes(G,0,'penwidth')
    nx.set_node_attributes(G,dict_month_giver,'penwidth')
    
    # draw the graph
    p=nx.drawing.nx_pydot.to_pydot(G)
    p.set_ratio(.3)
    p.set_size(150)

    p.write_png('visualization/monthlyflow/month-%s.png'%idx.month)

/Users/zhiweili/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning:

divide by zero encountered in log



### finer-grain: separate near v.s. non-near daos

In [10]:
df_daoinfo = pd.read_csv('data/Data analysis - DAO Info.csv')
#df_daoinfo['Name'] = df_daoinfo['Name'].str.lower()
df_daoinfo['Name'] = df_daoinfo['Name'].str.replace('-', '').str.lower()
df_daoinfo['Mission tied to Near ecosystem growth'] = df_daoinfo['Mission tied to Near ecosystem growth'].str.lower()

df_daoinfo.head()

,Name,Mission,Link,Mission tied to Near ecosystem growth
0,afrikaburn,Afrika Burn music / art festival,https://www.mintbase.io/store/afrikaburn.mintb...,no
1,appalabs,Development of Appa Labs and Growth of the App...,https://www.sputnik.fund/#/dao/appalabs.sputni...,no
2,arrozcriativo,Creating a NEAR DAO for Arroz Estudios & integ...,https://www.sputnik.fund/#/dao/arrozcriativo.s...,yes
3,assemblyscript,"For the development of ""assembly script"", a sm...",NaN,yes
4,art,"Creativity development, support and research o...",https://www.sputnik.fund/#/dao/art.sputnikdao....,no


In [12]:
# use the near/non-near categorization to color each node
dic_daocolor = dict()
for key in df_daoinfo['Name'].values:
    isnear = df_daoinfo['Mission tied to Near ecosystem growth'].loc[df_daoinfo['Name']==key].values[0]
    if isnear=='yes':
        dic_daocolor[key] ='#998CEB'
    elif isnear=='no':
        dic_daocolor[key] ='#77E4D4'
#     else: # unknowm
#         dic_daocolor[key] ='#FAFAFA'
     
                    


In [125]:
# now plot
for idx, df_month_flow in df_time_flow.groupby(level='time'): 
    df_month_flow.reset_index(inplace=True) # convert the multi-index(from grouping) into columns
    df_month_flow['weight']= np.log(df_month_flow['amount'])
    # Dropping all the rows with 0 funding amount values
    df_month_flow.replace([np.inf, -np.inf], np.nan, inplace=True)  
    df_month_flow.dropna(inplace=True)
    # set the edge width
    df_month_flow['penwidth']= .5+4*(df_month_flow['weight'] - min(df_month_flow['weight']))/(max(df_month_flow['weight'])-min(df_month_flow['weight']))
    
    G=nx.from_pandas_edgelist(df_month_flow, 'dao_give', "dao_receive", edge_attr=True,create_using=nx.DiGraph())
    
    
    # define node attributes
    dict_month_giver=np.log(df_month_flow.groupby('dao_give').sum()).to_dict()['amount']
    #nx.set_node_attributes(G,min(dict_month_giver),'size')
    nx.set_node_attributes(G,0,'penwidth')
    nx.set_node_attributes(G,dict_month_giver,'penwidth')
    nx.set_node_attributes(G,dic_daocolor,name="fillcolor")
    nx.set_node_attributes(G,'filled',name="style")
    

    # draw the graph
    p=nx.drawing.nx_pydot.to_pydot(G)
    p.set_ratio(.3)
    p.set_size(150)

    p.write_png('visualization/monthlyflow/categorized-month-%s.png'%idx.month)

/Users/zhiweili/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning:

divide by zero encountered in log



#### quantify this trend: how much of funding goes to the non-near daos every month?

In [128]:
df_cleantrans.head()

,dao_give,dao_receive,amount,time
0,communitysquad,createbase,300.0,2021-04-20 18:19:21.470696448
1,communitysquad,portuguese,150.0,2021-05-13 19:41:04.447497216
2,communitysquad,sandbox,2551.0,2021-05-19 15:22:09.020133120
3,communitysquad,nftgaming,2000.0,2021-05-20 16:12:46.214436608
4,communitysquad,vr,2000.0,2021-05-20 16:16:08.911319040


In [28]:

affiliation = []
for krow,row in df_cleantrans.iterrows():
    isnear = df_daoinfo['Mission tied to Near ecosystem growth'].loc[df_daoinfo['Name']==row['dao_receive']]
    if len(isnear)==0:
        print(row['dao_receive']) # daos that doesn't have affiliation info
        affiliation.append(np.nan)
        continue
    else:
        isnear = isnear.values[0]
        #breakpoint()
    if isnear=='yes':
        affiliation.append(0)
    else:
        affiliation.append(1)
df_cleantrans['is_receiver_nonneardao'] = affiliation        
df_cleantrans['is_receiver_neardao'] = 1-np.array(affiliation)

china_charity_dao
sanfrancisconearhacknode
sanfrancisconearhacknode
ceramic
alas
near_codame_china
tb


/Users/zhiweili/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/zhiweili/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [29]:
df_cleantrans.head()

,dao_give,dao_receive,amount,time,is_receiver_nonneardao,is_receiver_neardao
0,communitysquad,createbase,300.0,2021-04-20 18:19:21.470696448,0.0,1.0
1,communitysquad,portuguese,150.0,2021-05-13 19:41:04.447497216,0.0,1.0
2,communitysquad,sandbox,2551.0,2021-05-19 15:22:09.020133120,0.0,1.0
3,communitysquad,nftgaming,2000.0,2021-05-20 16:12:46.214436608,0.0,1.0
4,communitysquad,vr,2000.0,2021-05-20 16:16:08.911319040,0.0,1.0


In [25]:
df_month_plot.head()

,dao_give,dao_receive,amount,time,non-near daos
0,communitysquad,createbase,300.0,2021-04-20 18:19:21.470696448,0.0
1,communitysquad,portuguese,150.0,2021-05-13 19:41:04.447497216,0.0
2,communitysquad,sandbox,2551.0,2021-05-19 15:22:09.020133120,0.0
3,communitysquad,nftgaming,2000.0,2021-05-20 16:12:46.214436608,0.0
4,communitysquad,vr,2000.0,2021-05-20 16:16:08.911319040,0.0


In [31]:
df_month_plot=df_cleantrans.dropna()
df_month_plot = df_month_plot.rename(columns={'is_receiver_nonneardao':'non-near daos','is_receiver_neardao':'near daos'})
fig3 = px.histogram(df_month_plot,x='time',y=['non-near daos','near daos'],histfunc='sum',title='total number of daos receiving funds')

In [32]:
fig3.write_image("visualization/fig3_Nreceiver_categorized.png")  


# Overall comparison of NEAR v.s. non-NEAR DAOs

In [15]:
# load the current value of each dao
df_daostats = pickle.load(open('dao_stats','rb'))
df_daostats['Name']=df_daostats['Name'].replace('.sputnikdao.near','',regex=True).replace('-','',regex=True).replace(' ','',regex=True).str.lower()

df_daostats.head()

,Name,Ⓝ Value,Value (USD),Proposals,In Progress,Successful,Failed,Expired
0,genesis,15073.0,157965.0,45.0,0.0,15.0,9.0,21.0
1,academy,48.0,503.0,5.0,0.0,2.0,0.0,3.0
2,ethdenver,264.0,2767.0,133.0,0.0,92.0,12.0,28.0
3,arts,205.0,2144.0,8.0,0.0,7.0,0.0,1.0
4,createbase,503.0,5272.0,483.0,4.0,370.0,77.0,22.0


In [16]:
Stats = {'near_value':[],'usd_value':[],'total_proposal':[],'successful_proposal':[]}
for krow,row in df_daoinfo.iterrows():
    basicinfo = df_daostats.loc[df_daostats['Name']==row['Name']]
    if len(basicinfo)==0:
        if row['Name']=='codame':
            basicinfo = df_daostats.loc[df_daostats['Name']=='near_codame_china']
        elif row['Name']=='china_chartity_dao':
            basicinfo = df_daostats.loc[df_daostats['Name']=='china_charity_dao']
        elif row['Name']=='ecosystem(kog)':
            basicinfo = df_daostats.loc[df_daostats['Name']=='knightsofgondor']
        elif row['Name']=='hype':
            basicinfo = df_daostats.loc[df_daostats['Name']=='nfthypesquad']
        elif row['Name']=='sanfranciscohacknode':
            basicinfo = df_daostats.loc[df_daostats['Name']=='sanfrancisconearhacknode']
        
        elif (row['Name']=='dj') or (row['Name']=='drops') or (row['Name']=='vr'):
            basicinfo = df_daostats.loc[df_daostats['Name']==row['Name']+'dao']
        else:
            print(row['Name'])
            Stats['near_value'].append(np.nan)
            Stats['usd_value'].append(np.nan)
            Stats['total_proposal'].append(np.nan)
            Stats['successful_proposal'].append(np.nan)
            continue
    
    Stats['near_value'].append(float(basicinfo['Ⓝ Value']))
    Stats['usd_value'].append(float(basicinfo['Value (USD)']))
    Stats['total_proposal'].append(float(basicinfo['Proposals']))
    Stats['successful_proposal'].append(float(basicinfo['Successful']))
    

afrikaburn
curators
lisboanode
tomadao
wiki


the non-existing ones could be the receivers in astro. just ignore them for now

In [17]:
df_daoinfo_more = pd.concat([df_daoinfo,pd.DataFrame(Stats)],axis=1)

In [229]:
df_daoinfo_more.head()

,Name,Mission,Link,Mission tied to Near ecosystem growth,near_value,usd_value,total_proposal,successful_proposal
0,afrikaburn,Afrika Burn music / art festival,https://www.mintbase.io/store/afrikaburn.mintb...,no,NaN,NaN,NaN,NaN
1,appalabs,Development of Appa Labs and Growth of the App...,https://www.sputnik.fund/#/dao/appalabs.sputni...,no,7.0,68.0,7.0,4.0
2,arrozcriativo,Creating a NEAR DAO for Arroz Estudios & integ...,https://www.sputnik.fund/#/dao/arrozcriativo.s...,yes,262.0,2744.0,93.0,88.0
3,assemblyscript,"For the development of ""assembly script"", a sm...",NaN,yes,6403.0,67103.0,21.0,14.0
4,art,"Creativity development, support and research o...",https://www.sputnik.fund/#/dao/art.sputnikdao....,no,6.0,58.0,18.0,17.0


In [234]:
df_daoinfo_more_cleaned

,Name,Mission,Link,Mission tied to Near ecosystem growth,near_value,usd_value,total_proposal,successful_proposal
1,appalabs,Development of Appa Labs and Growth of the App...,https://www.sputnik.fund/#/dao/appalabs.sputni...,no,7.0,68.0,7.0,4.0
2,arrozcriativo,Creating a NEAR DAO for Arroz Estudios & integ...,https://www.sputnik.fund/#/dao/arrozcriativo.s...,yes,262.0,2744.0,93.0,88.0
3,assemblyscript,"For the development of ""assembly script"", a sm...",NaN,yes,6403.0,67103.0,21.0,14.0
4,art,"Creativity development, support and research o...",https://www.sputnik.fund/#/dao/art.sputnikdao....,no,6.0,58.0,18.0,17.0
5,berryclub,NEAR Berryclub 中文行会(微信公众号：链资产，chainassets)围绕NE...,https://www.sputnik.fund/#/dao/berryclub.sputn...,yes,228.0,2392.0,84.0,15.0
6,bogotahacknode,"Host a Hack Node in Bogota, the capital city o...",https://www.sputnik.fund/#/dao/bogotahacknode....,yes,5.0,52.0,1.0,1.0
7,c1,To establish a network of charitable art facil...,https://www.sputnik.fund/#/dao/c1.sputnikdao.near,yes,6.0,58.0,36.0,31.0
8,china_chartity_dao,NEAR公益募捐基金，用于NEAR基金会以及NEAR社区慈善捐赠；艺术品NFT慈善拍卖款，以...,https://www.sputnik.fund/#/dao/china_charity_d...,yes,20129.0,210948.0,5.0,4.0
9,clan,The Clan DAO exists to help artists on the run...,https://www.sputnik.fund/#/dao/clan.sputnikdao...,no,3.0,37.0,14.0,14.0
10,codame,"Chinese Artists NFT onboarding, encouraging ac...",https://www.sputnik.fund/#/search/dao/codame,no,8.0,86.0,21.0,14.0


In [18]:
df_daoinfo_more_cleaned = df_daoinfo_more.dropna(subset=['near_value','total_proposal',"Mission tied to Near ecosystem growth"])
fig = px.scatter(df_daoinfo_more_cleaned, y="Mission tied to Near ecosystem growth", x="successful_proposal",hover_data=['Name'],
                 size=np.log(df_daoinfo_more_cleaned["near_value"]),color="Mission tied to Near ecosystem growth",
                log_x=False)


In [33]:
fig.show()
fig.write_image("visualization/fig4_dao_overview_categorized.png")  


In [34]:
df_daoinfo_more_cleaned = df_daoinfo_more.dropna(subset=['near_value',"Mission tied to Near ecosystem growth","total_proposal"])
fig = px.scatter(df_daoinfo_more_cleaned, y="successful_proposal", x="near_value",hover_data=['Name'],
                 size="total_proposal",color="Mission tied to Near ecosystem growth",
                log_x=True,title='Sputnik DAOs overview')


In [36]:
fig.write_image("visualization/fig5_dao_overview_alt_categorized.png")  


In [35]:
fig.show()

## who are the top non-near daos?

### in terms of near-value

In [281]:
df_daoinfo_more_cleaned.loc[df_daoinfo_more_cleaned['Mission tied to Near ecosystem growth']=='no'].sort_values(by='near_value',ascending=False)[:10]

,Name,Mission,Link,Mission tied to Near ecosystem growth,near_value,usd_value,total_proposal,successful_proposal
53,rucommunity,Create awareness and support various initiativ...,https://www.sputnik.fund/#/dao/rucommunity.spu...,no,2109.0,22098.0,90.0,81.0
64,zenbikers,The ZEN Bikers Guild is a thematic guild with ...,https://www.sputnik.fund/#/dao/zenbikers.sputn...,no,523.0,5482.0,7.0,5.0
39,muti,Digital equivalent to the physical collective ...,https://www.sputnik.fund/#/dao/muti.sputnikdao...,no,386.0,4043.0,113.0,108.0
28,indiacovidrelief,This DAO will act as a charitable entity for h...,https://www.sputnik.fund/#/dao/indiacovidrelie...,no,144.0,1509.0,2.0,1.0
44,nearch4n,Bring NEARCH4N to life through community input...,https://www.sputnik.fund/#/dao/nearch4n.sputni...,no,101.0,1056.0,18.0,18.0
36,motion,MotionDAO offers to dance-tech.net 1 members a...,https://www.sputnik.fund/#/dao/motion.sputnikd...,no,75.0,791.0,13.0,10.0
20,drops,The Drops DAO is the vehicle for the DAOdrops ...,https://www.sputnik.fund/#/dao/drops-dao.sputn...,no,60.0,630.0,12.0,11.0
19,dj,DJ DAO is for the growth of the DJ community i...,https://www.sputnik.fund/#/dao/dj-dao.sputnikd...,no,56.0,587.0,4.0,4.0
24,graffiti,Supporting a community of Graffiti Artists acr...,https://www.sputnik.fund/#/dao/graffiti.sputni...,no,20.0,210.0,5.0,5.0
10,codame,"Chinese Artists NFT onboarding, encouraging ac...",https://www.sputnik.fund/#/search/dao/codame,no,8.0,86.0,21.0,14.0


### in terms of total successful proposals

In [282]:
df_daoinfo_more_cleaned.loc[df_daoinfo_more_cleaned['Mission tied to Near ecosystem growth']=='no'].sort_values(by='successful_proposal',ascending=False)[:10]

,Name,Mission,Link,Mission tied to Near ecosystem growth,near_value,usd_value,total_proposal,successful_proposal
39,muti,Digital equivalent to the physical collective ...,https://www.sputnik.fund/#/dao/muti.sputnikdao...,no,386.0,4043.0,113.0,108.0
27,incubadorapt,"DAO that aims to onboard artists and curators,...",https://www.sputnik.fund/#/dao/incubadorapt.sp...,no,6.0,63.0,86.0,83.0
53,rucommunity,Create awareness and support various initiativ...,https://www.sputnik.fund/#/dao/rucommunity.spu...,no,2109.0,22098.0,90.0,81.0
14,cudo,"CUDO DAO - IMMIGRANT AUDIOVISUAL, made by Immi...",https://www.sputnik.fund/#/dao/cudo.sputnikdao...,no,5.0,56.0,76.0,72.0
15,cudo,"CUDO DAO - IMMIGRANT AUDIOVISUAL, made by Immi...",https://www.sputnik.fund/#/dao/cudo.sputnikdao...,no,5.0,56.0,76.0,72.0
44,nearch4n,Bring NEARCH4N to life through community input...,https://www.sputnik.fund/#/dao/nearch4n.sputni...,no,101.0,1056.0,18.0,18.0
4,art,"Creativity development, support and research o...",https://www.sputnik.fund/#/dao/art.sputnikdao....,no,6.0,58.0,18.0,17.0
9,clan,The Clan DAO exists to help artists on the run...,https://www.sputnik.fund/#/dao/clan.sputnikdao...,no,3.0,37.0,14.0,14.0
10,codame,"Chinese Artists NFT onboarding, encouraging ac...",https://www.sputnik.fund/#/search/dao/codame,no,8.0,86.0,21.0,14.0
61,vitalpointai,To coordinate and fund Vital Point AI projects.,https://www.sputnik.fund/#/dao/vitalpointai.sp...,no,5.0,48.0,14.0,11.0


## metrics check: is "total proposal" and "successful proposal" basically correlated?

In [283]:
fig = px.scatter(df_daoinfo_more_cleaned, y="successful_proposal", x="total_proposal",hover_data=['Name'])


In [284]:
fig.show()

yep, very much